<a href="https://colab.research.google.com/github/rafiqmyura/Learn-Machine-Learning/blob/main/Flower_Detection_Type_(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import time
import shutil
import pathlib
import itertools
from PIL import Image


# import data handling tools

import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
# import Deep Learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

# Warning Notification

import warnings
warnings.filterwarnings("ignore")

print('modules loaded')

modules loaded


## Data Preprocessing

**Read DataFrame and Store**

In [9]:
data_dir= '/content/drive/MyDrive/frayer data/flower_images'
filepath = []
labels = []

folds = os.listdir(data_dir)
for fold in folds :
  foldpath = os.path.join(data_dir, fold)
  filelist = os.listdir(foldpath)
  for file in filelist:
    fpath = os.path.join(foldpath, file)
    filepath.append(fpath)
    labels.append(fold)


Fseries = pd.Series(filepath, name= 'filepath')
Lseries = pd.Series(labels, name='labels')
df = pd.concat([Fseries, Lseries], axis=1)

In [10]:
df

,filepath,labels
0,/content/drive/MyDrive/frayer data/flower_imag...,Lilly
1,/content/drive/MyDrive/frayer data/flower_imag...,Lilly
2,/content/drive/MyDrive/frayer data/flower_imag...,Lilly
3,/content/drive/MyDrive/frayer data/flower_imag...,Lilly
4,/content/drive/MyDrive/frayer data/flower_imag...,Lilly
...,...,...
4995,/content/drive/MyDrive/frayer data/flower_imag...,Tulip
4996,/content/drive/MyDrive/frayer data/flower_imag...,Tulip
4997,/content/drive/MyDrive/frayer data/flower_imag...,Tulip
4998,/content/drive/MyDrive/frayer data/flower_imag...,Tulip


In [13]:
train_df, dummy_df = train_test_split(df, train_size=0.8, shuffle=True, random_state=123)


valid_df, test_df = train_test_split(dummy_df,  train_size= 0.6, shuffle= True, random_state= 123)

In [17]:
batch_size = 16
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()
train_gen = tr_gen.flow_from_dataframe( train_df, x_col= 'filepath', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                       color_mode= 'rgb', shuffle= True, batch_size= batch_size)

valid_gen = ts_gen.flow_from_dataframe( valid_df, x_col= 'filepath', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= True, batch_size= batch_size)

test_gen = ts_gen.flow_from_dataframe( test_df, x_col= 'filepath', y_col= 'labels', target_size= img_size, class_mode= 'categorical',
                                    color_mode= 'rgb', shuffle= False, batch_size= batch_size)

Found 4000 validated image filenames belonging to 5 classes.
Found 600 validated image filenames belonging to 5 classes.
Found 400 validated image filenames belonging to 5 classes.
